<a href="https://colab.research.google.com/github/Andreaierardi/SocialNetworkAnalysis-project/blob/main/project_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Social Network Analysis Project

### Università degli Studi di Milano 



---



In [ ]:
# Nodes 
# https://github.com/Andreaierardi/SocialNetworkAnalysis-project/blob/main/dataset/fb-pages-food.csv

# Edges
# https://github.com/Andreaierardi/SocialNetworkAnalysis-project/blob/main/dataset/fb-pages-food.edges

In [30]:
import pandas as pd
nodes = pd.read_csv('https://raw.githubusercontent.com/Andreaierardi/SocialNetworkAnalysis-project/main/dataset/fb-pages-food.csv?token=AIWS4RUL6EGKERX23BLXBGK7XPQ3E')

edges = pd.read_csv('https://raw.githubusercontent.com/Andreaierardi/SocialNetworkAnalysis-project/main/dataset/fb-pages-food.edges?token=AIWS4RQCC5WOTZKTUZTINKC7XPQYG')


ParserError: ignored

In [ ]:
print(nodes)
print(edges)